<a href="https://colab.research.google.com/github/HaoCuii/Calculator-11/blob/main/find_the_yum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

import os, warnings
import matplotlib.pyplot as plt
from matplotlib import gridspec

from tensorflow import keras
from keras import layers
from keras.applications import VGG16

import numpy as np
import tensorflow as tf
from keras.preprocessing import image_dataset_from_directory
from sklearn.preprocessing import LabelEncoder
from keras.callbacks import EarlyStopping

from PIL import Image
import torch
from torchvision import transforms


plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)
plt.rc('image', cmap='magma')

tf.config.run_functions_eagerly(True)

FRUIT DATA CREATING CSV


In [ ]:
images_train = []
labels_train = []

for i in range(len(train_filenames)):
    try:
      img_name = train_filenames[i]
      label = train_categories[i]

      image = Image.open(img_name).convert('RGB')
      image = image.resize((224,224))
      image_array = np.array(image)

      images_train.append(image_array)
      labels_train.append(label)
    except:
      pass

np_images_train = np.array(images_train)
np_labels_train = np.array(labels_train)


In [ ]:
from sklearn.utils import shuffle

np_images_train = np_images_train.reshape(np_images_train.shape[0], -1)
X_train = pd.DataFrame(np_images_train)

np_labels_train = np_labels_train.reshape(np_labels_train.shape[0], -1)
y_train = pd.DataFrame(np_labels_train)

X_train, y_train = shuffle(X_train, y_train, random_state=42)


'''np_images_test = np_images_test.reshape(np_images_test.shape[0], -1)
X_test = pd.DataFrame(np_images_test)

np_labels_test = np_labels_test.reshape(np_labels_test.shape[0], -1)
y_test = pd.DataFrame(np_labels_test)

X_test, y_test = shuffle(X, y, random_state=42)'''

In [ ]:
label_encoder = LabelEncoder()
y_encoded_train = label_encoder.fit_transform(y_train.values.ravel())  # Ensure y is 1D
'''y_encoded_test = label_encoder.fit_transform(y_test.values.ravel())  # Ensure y is 1D'''

In [ ]:
batch_size = X_train.shape[0]  # Number of samples
height = 224  # Original height of images
width = 224   # Original width of images
channels = 3  # Number of channels (e.g., 3 for RGB)

# Reshape the flattened images back to 3D shape
X_reshaped_train = X_train.values.reshape(batch_size, height, width, channels)

# Now you can resize the images
X_resized_train = tf.image.resize(X_reshaped_train, (224, 224))
X_resized_train = X_resized_train.numpy().astype('float32')  # Ensure dtype is float32
# Normalize pixel values to [0, 1]
X_resized_train = X_resized_train / 255.0

# Now X_resized is ready for training or prediction
print(X_resized_train.shape)  # Should print (batch_size, 224, 224, 3)

X_train = X_resized_train

'''
batch_size = X_test.shape[0]  # Number of samples

X_resized_test = X_test.values.reshape(batch_size, height, width, channels)
X_resized_test = tf.image.resize(X_resized_test, (224, 224))
X_resized_test = X_resized_test.numpy().astype('float32')
X_resized_test = X_resized_test / 255.0

X_test = X_resized_test'''

In [ ]:
base_model = VGG16(weights='imagenet')
base_model.trainable = False

model = keras.Sequential([
    #base_model,
    layers.Flatten(),
    layers.BatchNormalization(input_shape=(X_train.shape[1],)),

    layers.Dense(1024, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),

    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),

    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),

    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),

    layers.Dense(len(label_encoder.classes_), activation='softmax'),  # Matches # classes
])

In [ ]:
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=30,
    restore_best_weights=True,
)

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(epsilon=0.01),
    loss='sparse_categorical_crossentropy',  # Integer labels
    metrics=['accuracy'],
)

In [ ]:
history = model.fit(
    X_train, y_train,
    #validation_data=(), ########################################## CHANGE HERE
    batch_size=128,  # Smaller batch size
    epochs=256,
    validation_split=0.2,
    callbacks=[early_stopping],
)

history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot(title="Cross-Entropy Loss")
history_df.loc[:, ['accuracy', 'val_accuracy']].plot(title="Accuracy")

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the image and preprocess
img_path = "/content/drive/My Drive/regen_fruit_dataset/train/Banana/img_1371.jpeg"

# Load the image (resize it to 128x128 as expected by the model)
img = image.load_img(img_path, target_size=(224, 224))

# Convert the image to a numpy array
img_array = image.img_to_array(img)

# Add the batch dimension
img_array = np.expand_dims(img_array, axis=0)  # This will change the shape to (1, 128, 128, 3)

# Normalize the image if needed (based on how you processed it during training)
img_array = img_array / 255.0  # If you scaled during training

# Now you can use this img_array to make predictions
predictions = model.predict(img_array)

# If you're doing classification, find the class with the highest probability
predicted_class = np.argmax(predictions, axis=1)

# If you have the list of labels for classes
fruit_categories = [
    'Apple',
    'Banana',
    'avocado',
    'cherry',
    'kiwi',
    'mango',
    'orange',
    'pinenapple',
    'strawberries',
    'watermelon']
predicted_label = fruit_categories[predicted_class[0]]

print(f"Predicted class: {predicted_label}")


In [ ]:
model.save('ReGen_CNN.keras')